In [2]:
import pandas as pd

In [4]:
df = pd.read_sas('individus_ct2013.sas7bdat', encoding='latin-1')
df.head()

,C1C,C1D,IDENT_MEN,IDENT_IND,NPERS,NACTIFS,TYPMEN5,TYPMEN15,IR,REVMEN,...,lnaisd,lnaisper,lnaismer,natnaisper,natnaismer,nati,revannu_drap,revannu,revmensc,revmenscc_drap
0,2,2,0100019101000,010001910100002,2.0,1.0,2,22,1,2200.0,...,11,11,11,NaN,NaN,NaN,1.0,1500.0,125.000000,1.0
1,NaN,NaN,0100083001000,010008300100002,2.0,2.0,5,52,2,2250.0,...,11,11,11,NaN,NaN,NaN,-2.0,NaN,NaN,-2.0
2,2,2,0100084601000,010008460100002,5.0,2.0,4,42,1,2300.0,...,11,11,11,NaN,NaN,NaN,1.0,0.0,0.000000,1.0
3,2,2,0100091701000,010009170100001,2.0,1.0,3,31,4,NaN,...,11,11,11,NaN,NaN,NaN,1.0,3700.0,308.333333,1.0
4,1,1,0100091801000,010009180100001,1.0,1.0,1,10,1,1200.0,...,11,11,11,NaN,NaN,NaN,1.0,15000.0,1250.000000,1.0
